# PydanticOutputParser

- PydanticOutputParser (이는 대부분의 OutputParser 에 해당되기도 합니다) 에는 주로 두 가지 핵심 메서드가 구현 되어야 합니다.

1. `get_format_instructions()`: 언어 모델이 출력해야 할 정보의 형식을 정의하는 지침(instruction) 을 제공합니다. 예를 들어, 언어 모델이 출력해야 할 데이터의 필드와 그 형태를 설명하는 지침을 문자열로 반환할 수 있습니다. 이때 설정하는 지침(instruction) 의 역할이 매우 중요합니다. 이 지침에 따라 언어 모델은 출력을 구조화하고, 이를 특정 데이터 모델에 맞게 변환할 수 있습니다.

2. `parse()`: 언어 모델의 출력(문자열로 가정)을 받아들여 이를 특정 구조로 분석하고 변환합니다. Pydantic와 같은 도구를 사용하여, 입력된 문자열을 사전 정의된 스키마에 따라 검증하고, 해당 스키마를 따르는 데이터 구조로 변환합니다.

처음은 늘 import

In [2]:
from dotenv import load_dotenv

load_dotenv()


True

출력 결과 분석을 위해 langsmith

In [3]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH03-OutputParser-Junu")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH03-OutputParser-Junu


In [4]:
# 실시간 출력을 위한 import
from langchain_teddynote.messages import stream_response

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field


llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

이메일 본문 예시

In [6]:
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""


##### 출력 파서를 사용하지 않는 경우 예시
-> 실행할 때 마다 출력 결과가 변하고 내용도 일관되지 않다.

In [9]:
from itertools import chain
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    "다음의 이메일 내용중 중요한 내용을 추출해 주세요.\n\n{email_conversation}"
)

llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

chain = prompt | llm

answer = chain.stream({"email_conversation": email_conversation})

output = stream_response(answer, return_output=True)

이메일의 중요한 내용은 다음과 같습니다:

1. 김철수 상무는 바이크코퍼레이션 소속이며, 이은채 대리에게 이메일을 보냈습니다.
2. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 활동하는 기업입니다.
3. 김철수 상무는 "ZENESIS" 자전거에 대한 상세한 브로슈어를 요청하며, 특히 기술 사양, 배터리 성능, 디자인 측면의 정보를 필요로 합니다.
4. 협력 가능성을 논의하기 위해 1월 15일 화요일 오전 10시에 미팅을 제안했습니다.
5. 미팅 장소는 이은채 대리가 근무하는 회사의 사무실입니다.

참고로, Field 안에 description 은 텍스트 형태의 답변에서 주요 정보를 추출하기 위한 설명입니다. LLM 이 바로 이 설명을 보고 필요한 정보를 추출하게 됩니다. 그러므로 이 설명은 정확하고 명확해야 합니다.

##### Pydantic을 활용한 파서

In [10]:
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")


# PydanticOutputParser 생성
parser = PydanticOutputParser(pydantic_object=EmailSummary)

In [11]:
# instruction 을 출력합니다.
print(parser.get_format_instructions())


The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"description": "메일을 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "메일을 보낸 사람의 이메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type": "string"}, "date": {"description": "메일 본문에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}
```


프롬프트를 정의합니다.

1. `question`: 유저의 질문을 받습니다.
2. `email_conversation`: 이메일 본문의 내용을 입력합니다.
3. `format`: 형식을 지정합니다.

In [12]:
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

# format 에 PydanticOutputParser의 부분 포맷팅(partial) 추가
prompt = prompt.partial(format=parser.get_format_instructions())


다음으로는 Chain 을 생성합니다.

In [13]:
# chain 을 생성합니다.
chain = prompt | llm


체인을 실행하고 결과를 확인합니다.

In [14]:
# chain 을 실행하고 결과를 출력합니다.
response = chain.stream(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

# 결과는 JSON 형태로 출력됩니다.
output = stream_response(response, return_output=True)

```json
{
    "person": "김철수",
    "email": "chulsoo.kim@bikecorporation.me",
    "subject": "\"ZENESIS\" 자전거 유통 협력 및 미팅 일정 제안",
    "summary": "김철수 상무는 바이크코퍼레이션의 자전거 유통 협력을 위해 ZENESIS 모델의 상세 브로슈어를 요청하고, 1월 15일 화요일 오전 10시에 미팅을 제안합니다.",
    "date": "1월 15일 화요일 오전 10시"
}
```

마지막으로 parser를 사용하여 결과를 파싱하고 EmailSummary 객체로 변환합니다.

In [15]:
# PydanticOutputParser 를 사용하여 결과를 파싱합니다.
structured_output = parser.parse(output)
print(structured_output)

person='김철수' email='chulsoo.kim@bikecorporation.me' subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안' summary='김철수 상무는 바이크코퍼레이션의 자전거 유통 협력을 위해 ZENESIS 모델의 상세 브로슈어를 요청하고, 1월 15일 화요일 오전 10시에 미팅을 제안합니다.' date='1월 15일 화요일 오전 10시'


### parser 가 추가된 체인 생성

출력 결과를 정의한 Pydantic 객체로 생성할 수 있습니다.

In [16]:
# 출력 파서를 추가하여 전체 체인을 재구성합니다.
chain = prompt | llm | parser

In [17]:
# chain 을 실행하고 결과를 출력합니다.
response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

# 결과는 EmailSummary 객체 형태로 출력됩니다.
response


EmailSummary(person='김철수', email='chulsoo.kim@bikecorporation.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary='김철수 상무는 바이크코퍼레이션에서 ZENESIS 자전거의 브로슈어를 요청하며, 기술 사양, 배터리 성능, 디자인 정보가 필요하다고 언급했습니다. 또한, 협력 가능성을 논의하기 위해 1월 15일 화요일 오전 10시에 미팅을 제안했습니다.', date='1월 15일 화요일 오전 10시')

### with_structured_output()

.with_structured_output(Pydantic)을 사용하여 출력 파서를 추가하면, 출력을 Pydantic 객체로 변환할 수 있습니다.

In [18]:
llm_with_structered = ChatOpenAI(
    temperature=0, model_name="gpt-4o"
).with_structured_output(EmailSummary)


In [19]:
# invoke() 함수를 호출하여 결과를 출력합니다.
answer = llm_with_structered.invoke(email_conversation)
answer


EmailSummary(person='김철수', email='chulsoo.kim@bikecorporation.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary='김철수 상무는 이은채 대리에게 바이크코퍼레이션과 테디인터내셔널 간의 협력 가능성을 논의하기 위해 이메일을 보냈습니다. 그는 테디인터내셔널의 새로운 자전거 모델 "ZENESIS"에 관심을 표명하며, 해당 모델의 기술 사양, 배터리 성능, 디자인에 대한 상세한 정보를 요청했습니다. 또한, 협력 논의를 위해 1월 15일 화요일 오전 10시에 미팅을 제안했습니다.', date='2023년 1월 8일')

한 가지 아쉬운 점은 .with_structured_output() 함수는 stream() 기능을 지원하지 않습니다.

# CommaSeparatedListOutputParser

CommaSeparatedListOutputParser 는 쉼표로 구분된 항목 목록을 반환할 필요가 있을 때 유용합니다.

In [20]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 콤마로 구분된 리스트 출력 파서 초기화
output_parser = CommaSeparatedListOutputParser()

# 출력 형식 지침 가져오기
format_instructions = output_parser.get_format_instructions()
# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    # 주제에 대한 다섯 가지를 나열하라는 템플릿
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],  # 입력 변수로 'subject' 사용
    # 부분 변수로 형식 지침 사용
    partial_variables={"format_instructions": format_instructions},
)

# ChatOpenAI 모델 초기화
model = ChatOpenAI(temperature=0)

# 프롬프트, 모델, 출력 파서를 연결하여 체인 생성
chain = prompt | model | output_parser

체인을 실행하고 결과를 출력합니다.

In [21]:
# 체인 호출 실행
chain.invoke({"subject": "역대 F1 드라이버 중 최고의 드라이버"})

['Juan Manuel Fangio',
 'Ayrton Senna',
 'Michael Schumacher',
 'Lewis Hamilton',
 'Alain Prost']

chain.stream을 사용하여 주제에 대한 스트림을 반복 처리합니다.
반복 중 스트림의 결과를 출력합니다.

In [24]:
# 스트림을 순회합니다.
for s in chain.stream({"subject": "오사카 관광명소 추천"}):
    print(s)  # 스트림의 내용을 출력합니다.

['오사카성']
['도톤보리']
['우메다 스카이 빌딩']
['신오사카']
['교토']


# StructuredOuputParser

 LLM에 대한 답변을 dict 형식으로 구성하고 key/value 쌍으로 갖는 여러 필드를 반환하고자 할 때 사용

In [25]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

- ResponseSchema 클래스를 사용하여 사용자의 질문에 대한 답변과 사용된 소스(웹사이트)에 대한 설명을 포함하는 응답 스키마를 정의합니다.
- StructuredOutputParser를 response_schemas를 사용하여 초기화하여, 정의된 응답 스키마에 따라 출력을 구조화합니다.

In [26]:
# 사용자의 질문에 대한 답변
response_schemas = [
    ResponseSchema(name="answer", description="사용자의 질문에 대한 답변"),
    ResponseSchema(
        name="source",
        description="사용자의 질문에 답하기 위해 사용된 `출처`, `웹사이트주소` 이여야 합니다.",
    ),
]
# 응답 스키마를 기반으로 한 구조화된 출력 파서 초기화
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)


이제 응답이 어떻게 포맷되어야 하는지에 대한 지시사항이 포함된 문자열을 받게 되며(schemas), 정의된 스키마를 프롬프트에 삽입합니다.

In [27]:
# 출력 형식 지시사항을 파싱합니다.
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    # 사용자의 질문에 최대한 답변하도록 템플릿을 설정합니다.
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    # 입력 변수로 'question'을 사용합니다.
    input_variables=["question"],
    # 부분 변수로 'format_instructions'을 사용합니다.
    partial_variables={"format_instructions": format_instructions},
)

In [28]:
model = ChatOpenAI(temperature=0)  # ChatOpenAI 모델 초기화
chain = prompt | model | output_parser  # 프롬프트, 모델, 출력 파서를 연결


In [30]:
# 대한민국의 수도가 무엇인지 질문합니다.
chain.invoke({"question": "역사상 가장 뛰어난 F1 드라이버는 누구인가요?"})


{'answer': '역사상 가장 뛰어난 F1 드라이버는 미하엘 쉼라인이라고 평가됩니다.',
 'source': 'https://www.formula1.com/en/drivers/hall-of-fame/Michael_Schumacher.html'}

chain.stream 메소드를 사용하여 "역사상 가장 뛰어난 F1 드라이버는 누구인가요?" 라는 질문에 대한 스트림 응답을 받습니다.

In [33]:
for s in chain.stream({"question": "역사상 가장 뛰어난 F1 드라이버는 누구인가요?"}):
    # 스트리밍 출력
    print(s)


{'answer': '역사상 가장 뛰어난 F1 드라이버는 미하엘 쉼라인이라고 평가됩니다.', 'source': 'https://www.formula1.com/en/drivers/hall-of-fame/Michael_Schumacher.html'}


# JsonOutputParser

In [34]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI


In [35]:
# OpenAI 객체를 생성합니다.
model = ChatOpenAI(temperature=0, model_name="gpt-4o")


In [36]:
# 원하는 데이터 구조를 정의합니다.
class Topic(BaseModel):
    description: str = Field(description="주제에 대한 간결한 설명")
    hashtags: str = Field(description="해시태그 형식의 키워드(2개 이상)")

In [37]:
# 질의 작성
question = "지구 온난화의 심각성 대해 알려주세요."

# 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입합니다.
parser = JsonOutputParser(pydantic_object=Topic)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

chain = prompt | model | parser  # 체인을 구성합니다.

chain.invoke({"question": question})  # 체인을 호출하여 쿼리 실행


{'description': '지구 온난화는 지구의 평균 기온이 상승하는 현상으로, 주로 인간의 활동에 의해 발생하는 온실가스 배출이 주요 원인입니다. 이는 극지방의 빙하가 녹고 해수면이 상승하며, 기후 패턴이 변화하는 등 심각한 환경적 영향을 초래합니다.',
 'hashtags': '#지구온난화 #기후변화 #환경보호'}

`JsonOutputParser`를 사용하여 파서를 설정하고, 프롬프트 템플릿에 지시사항을 주입합니다.

In [38]:
# 질의 작성
question = "현재 F1 상파울루 그랑프리 스프린트 결과 예측을 알려주세요."

# 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입합니다.
parser = JsonOutputParser(pydantic_object=Topic)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

chain = prompt | model | parser  # 체인을 구성합니다.

chain.invoke({"question": question})  # 체인을 호출하여 쿼리 실행


{'description': '상파울루 그랑프리 스프린트 레이스에서 맥스 페르스타펜이 우승할 가능성이 높습니다. 메르세데스와 페라리도 강력한 경쟁자로 예상됩니다.',
 'hashtags': '#F1 #상파울루그랑프리 #스프린트레이스'}

### Pydantic없이 사용하기

Pydantic 없이도 이 기능을 사용할 수 있습니다. 이 경우 JSON을 반환하도록 요청하지만, 스키마가 어떻게 되어야 하는지에 대한 구체적인 정보는 제공하지 않습니다.

In [41]:
# 질의 작성
question = "현재 F1 상파울루 그랑프리 스프린트 결과 예측을 알려주세요. 결과에 대한 설명은 `description`에, 관련 키워드는 `hashtags`에 담아주세요."

# JSON 출력 파서 초기화
parser = JsonOutputParser()

# 프롬프트 템플릿을 설정합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

# 지시사항을 프롬프트에 주입합니다.
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# 프롬프트, 모델, 파서를 연결하는 체인 생성
chain = prompt | model | parser

# 체인을 호출하여 쿼리 실행
response = chain.invoke({"question": question})

# 출력을 확인합니다.
print(response)


{'description': '상파울루 그랑프리 스프린트 레이스에서 맥스 페르스타펜이 우승할 것으로 예상됩니다. 그의 강력한 퍼포먼스와 레드불의 전략이 주요 요인입니다. 메르세데스와 페라리도 경쟁할 것으로 보이며, 날씨 변수도 결과에 영향을 미칠 수 있습니다.', 'hashtags': ['#F1', '#상파울루그랑프리', '#스프린트레이스', '#맥스페르스타펜', '#레드불', '#메르세데스', '#페라리']}


# PandasDataFrameOutputParser

In [42]:
import pprint
from typing import Any, Dict

import pandas as pd
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI


In [43]:
# ChatOpenAI 모델 초기화 (gpt-3.5-turbo 모델 사용을 권장합니다)
model = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")


format_parser_output 함수는 파서 출력을 사전 형식으로 변환하고 출력 형식을 지정하는 데 사용됩니다.

In [44]:
# 출력 목적으로만 사용됩니다.
def format_parser_output(parser_output: Dict[str, Any]) -> None:
    # 파서 출력의 키들을 순회합니다.
    for key in parser_output.keys():
        # 각 키의 값을 딕셔너리로 변환합니다.
        parser_output[key] = parser_output[key].to_dict()
    # 예쁘게 출력합니다.
    return pprint.PrettyPrinter(width=4, compact=True).pprint(parser_output)


- titanic.csv 데이터를 읽어온 뒤 DataFrame 을 로드하여 df 변수에 할당합니다.
- PandasDataFrameOutputParser를 사용하여 DataFrame을 파싱합니다.

In [46]:
# 원하는 Pandas DataFrame을 정의합니다.
df = pd.read_csv("./data/titanic.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [47]:
# 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입합니다.
parser = PandasDataFrameOutputParser(dataframe=df)

# 파서의 지시사항을 출력합니다.
print(parser.get_format_instructions())

The output should be formatted as a string as the operation, followed by a colon, followed by the column or row to be queried on, followed by optional array parameters.
1. The column names are limited to the possible columns below.
2. Arrays must either be a comma-separated list of numbers formatted as [1,3,5], or it must be in range of numbers formatted as [0..4].
3. Remember that arrays are optional and not necessarily required.
4. If the column is not in the possible columns or the operation is not a valid Pandas DataFrame operation, return why it is invalid as a sentence starting with either "Invalid column" or "Invalid operation".

As an example, for the formats:
1. String "column:num_legs" is a well-formatted instance which gets the column num_legs, where num_legs is a possible column.
2. String "row:1" is a well-formatted instance which gets row 1.
3. String "column:num_legs[1,2]" is a well-formatted instance which gets the column num_legs for rows 1 and 2, where num_legs is a p

컬럼에 대한 값을 조회하는 예제입니다.

In [48]:
# 열 작업 예시입니다.
df_query = "Age column 을 조회해 주세요."


# 프롬프트 템플릿을 설정합니다.
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],  # 입력 변수 설정
    partial_variables={
        "format_instructions": parser.get_format_instructions()
    },  # 부분 변수 설정
)

# 체인 생성
chain = prompt | model | parser

# 체인 실행
parser_output = chain.invoke({"query": df_query})

# 출력
format_parser_output(parser_output)

{'Age': {0: 22.0,
         1: 38.0,
         2: 26.0,
         3: 35.0,
         4: 35.0,
         5: nan,
         6: 54.0,
         7: 2.0,
         8: 27.0,
         9: 14.0,
         10: 4.0,
         11: 58.0,
         12: 20.0,
         13: 39.0,
         14: 14.0,
         15: 55.0,
         16: 2.0,
         17: nan,
         18: 31.0,
         19: nan}}


첫 번째 행을 검색하는 예시입니다.

In [49]:
# 행 조회 예시입니다.
df_query = "Retrieve the first row."

# 체인 실행
parser_output = chain.invoke({"query": df_query})

# 결과 출력
format_parser_output(parser_output)

{'0': {'Age': 22.0,
       'Cabin': nan,
       'Embarked': 'S',
       'Fare': 7.25,
       'Name': 'Braund, '
               'Mr. '
               'Owen '
               'Harris',
       'Parch': 0,
       'PassengerId': 1,
       'Pclass': 3,
       'Sex': 'male',
       'SibSp': 1,
       'Survived': 0,
       'Ticket': 'A/5 '
                 '21171'}}


특정 열에서 일부 행의 평균을 검색하는 작업 예제입니다.

In [50]:
# row 0 ~ 4의 평균 나이를 구합니다.
df["Age"].head().mean()


31.2

In [51]:
# 임의의 Pandas DataFrame 작업 예시, 행의 수를 제한합니다.
df_query = "Retrieve the average of the Ages from row 0 to 4."

# 체인 실행
parser_output = chain.invoke({"query": df_query})

# 결과 출력
print(parser_output)


{'mean': 31.2}


다음은 요금(Fare) 에 대한 평균 가격을 산정하는 예시입니다.

In [52]:
# 잘못 형식화된 쿼리의 예시입니다.
df_query = "Calculate average `Fare` rate."

# 체인 실행
parser_output = chain.invoke({"query": df_query})

# 결과 출력
print(parser_output)


{'mean': 22.19937}


In [53]:
# 결과 검증
df["Fare"].mean()


22.19937

# DatetimeOutputParser

In [54]:
from langchain.output_parsers import DatetimeOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 날짜 및 시간 출력 파서
output_parser = DatetimeOutputParser()
output_parser.format = "%Y-%m-%d"

# 사용자 질문에 대한 답변 템플릿
template = """Answer the users question:\n\n#Format Instructions: \n{format_instructions}\n\n#Question: \n{question}\n\n#Answer:"""

prompt = PromptTemplate.from_template(
    template,
    partial_variables={
        "format_instructions": output_parser.get_format_instructions()
    },  # 지침을 템플릿에 적용
)

# 프롬프트 내용을 출력
prompt

PromptTemplate(input_variables=['question'], input_types={}, partial_variables={'format_instructions': "Write a datetime string that matches the following pattern: '%Y-%m-%d'.\n\nExamples: 2025-11-08, 2024-11-08, 2025-11-07\n\nReturn ONLY this string, no other words!"}, template='Answer the users question:\n\n#Format Instructions: \n{format_instructions}\n\n#Question: \n{question}\n\n#Answer:')

In [55]:
# Chain 을 생성합니다.
chain = prompt | ChatOpenAI() | output_parser

# 체인을 호출하여 질문에 대한 답변을 받습니다.
output = chain.invoke({"question": "첫 F1경기가 개최된 연도는?"})


In [56]:
# 결과를 문자열로 변환
output.strftime("%Y-%m-%d")

'1950-05-13'

# EnumOutputParser

In [57]:
from langchain.output_parsers.enum import EnumOutputParser


- enum 모듈을 사용하여 Colors 클래스를 정의합니다.
- Colors 클래스는 Enum을 상속받으며, RED, GREEN, BLUE 세 가지 색상 값을 가집니다.

In [58]:
from enum import Enum


class Colors(Enum):
    RED = "빨간색"
    GREEN = "초록색"
    BLUE = "파란색"


In [59]:
# EnumOutputParser 인스턴스 생성
parser = EnumOutputParser(enum=Colors)


- 프롬프트에 사람의 정보({person})와 파싱 지침({instructions})을 포함시킵니다.
- parser.get_format_instructions() 함수를 호출하여 파싱 지침을 가져옵니다.
- 프롬프트, ChatOpenAI 모델, 파서를 연결하여 처리 체인을 구성합니다.

In [63]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 프롬프트 템플릿을 생성합니다.
prompt = PromptTemplate.from_template(
    """다음의 물체는 어떤 색깔인가요?

Object: {object}

Instructions: {instructions}"""
    # 파서에서 지시사항 형식을 가져와 부분적으로 적용합니다.
).partial(instructions=parser.get_format_instructions())
# 프롬프트와 ChatOpenAI, 파서를 연결합니다.
chain = prompt | ChatOpenAI() | parser


chain.invoke 함수를 사용하여 "원자"에 대한 정보를 요청합니다.

In [66]:
response = chain.invoke({"object": "원자"})  # "원자" 에 대한 체인 호출 실행
print(response)


OutputParserException: Response '원자는 초록색입니다.' is not one of the expected values: ['빨간색', '초록색', '파란색']
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

# OutputFixingParser

`OutputFixingParser`는 출력 파싱 과정에서 발생할 수 있는 오류를 자동으로 수정하는 기능을 제공합니다. 이 파서는 기본적으로 다른 파서, 예를 들어 PydanticOutputParser 를 래핑하고, 이 파서가 처리할 수 없는 형식의 출력이나 오류를 반환할 경우, 추가적인 LLM 호출을 통해 오류를 수정하도록 설계되었습니다.

이러한 접근 방식의 핵심은, 첫 번째 시도에서 스키마를 준수하지 않는 결과가 나올 경우, OutputFixingParser 가 자동으로 형식이 잘못된 출력을 인식하고, 이를 수정하기 위한 새로운 명령어와 함께 모델에 다시 제출한다는 것입니다. 이 과정에서, 수정을 위한 명령어는 오류를 정확히 지적하고, 올바른 형식으로 데이터를 재구성할 수 있도록 구체적인 지시를 포함해야 합니다.

예를 들어, PydanticOutputParser 를 사용하여 특정 데이터 스키마를 준수하는 출력을 생성하려고 했지만, 일부 필드가 누락되었거나 데이터 유형이 잘못된 경우가 발생할 수 있습니다. 이때 OutputFixingParser 는 다음 단계로, 해당 오류를 수정하는 지시를 포함한 새로운 요청을 LLM에 제출합니다. LLM은 이 지시를 바탕으로 오류를 수정한 새로운 출력을 생성하게 됩니다.

In [68]:
from langchain_openai import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List


class Actor(BaseModel):
    name: str = Field(description="name of an actor")
    film_names: List[str] = Field(
        description="list of names of films they starred in")


actor_query = "Generate the filmography for a random actor."

parser = PydanticOutputParser(pydantic_object=Actor)


In [69]:
# 잘못된 형식을 일부러 입력
misformatted = "{'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}"

# 잘못된 형식으로 입력된 데이터를 파싱하려고 시도
parser.parse(misformatted)

# 오류 출력

OutputParserException: Invalid json output: {'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

In [70]:
from langchain.output_parsers import OutputFixingParser

new_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI())

In [71]:
misformatted

"{'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}"

In [72]:
# OutputFixingParser 를 사용하여 잘못된 형식의 출력을 파싱
actor = new_parser.parse(misformatted)

In [73]:
# 파싱된 결과
actor

Actor(name='Tom Hanks', film_names=['Forrest Gump'])